# Web Scrapping de Yuhmak
#### Autor: Ciriondev
#### Este documento intenta mostrar el alcance del motor de webscraping desarrollado por la empresa, para ello se utilizará la página pública de Yuhmak para extraer las motocicletas catalogadas junto con los atributos descriptivos de cada publicación.
#### Ago 08, 2024

### Extracción de datos

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException

if 'google.colab' in str(get_ipython()):
    options = webdriver.ChromeOptions()
    options.add_argument('-headless')
    options.add_argument('-no-sandbox')
    options.add_argument('-disable-dev-shm-usage')

    driver = webdriver.Chrome('chromedriver',options=options)

else:
    service = Service(executable_path=r'/usr/local/bin/chromedriver')
    options = webdriver.ChromeOptions()
    options.add_argument('-headless')
    options.add_argument('-no-sandbox')
    options.add_argument('-disable-dev-shm-usage')
    options.add_argument("--window-size=1920,1200")

    driver = webdriver.Chrome(service=service,options=options)

Definimos la URL de la sección **Motos**:

In [7]:
driver.get("https://www.yuhmak.com/seccion-motos/")

Obtenemos el listado de todas las publicaciones disponibles en la página principal de la sección:

In [8]:
while True:
    try:
        ver = driver.find_element(By.LINK_TEXT, 'Mostrar más')
        ver.click()
        time.sleep(3)

    except NoSuchElementException as e:
        print(f"No se encontró elemento 'Mostrar más'")
        break

No se encontró elemento 'Mostrar más'


Encontramos los links a las publicaciones:

In [9]:
lista = driver.find_elements(By.CSS_SELECTOR, '.vtex-search-result-3-x-galleryItem.vtex-search-result-3-x-galleryItem--normal.vtex-search-result-3-x-galleryItem--grid.pa4')

In [10]:
links = []

for i in range(len(lista)):
    href = lista[i].find_element(By.CSS_SELECTOR, '.vtex-product-summary-2-x-clearLink.vtex-product-summary-2-x-clearLink--product-slider.h-100.flex.flex-column')
    links.append(href.get_attribute('href'))

In [11]:
len(links)

132

Procedemos a la recolección de los datos y su correspondiente análisis:

In [12]:
motos = pd.DataFrame()

for i in range(len(links) - 1):
        driver.get(links[i])
                
        try:
            titulo = driver.find_element(By.CLASS_NAME, 'yuhmak-yuhmak-installments-calculator-custom-0-x-productBrand ')
            tabla = driver.find_elements(By.CLASS_NAME, 'vtex-flex-layout-0-x-flexRow--productSpecificationItem')

            moto = dict({'Nombre': titulo.text})

            for i in range(len(tabla)):
                linea = tabla[i].text.split('\n')
                if len(linea) > 1:
                    atrib = linea[0]
                    valor = linea[1]
                    moto[atrib] = valor

            fila = pd.DataFrame([moto])
            motos = pd.concat([motos, fila], ignore_index=True)

        except NoSuchElementException as e:
            print(f"Error recolectando datos")

Error recolectando datos


Visualizamos la estructura de los datos obtenidos de manera dinámica:

In [19]:
motos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Nombre                    130 non-null    object
 1   Modelo                    57 non-null     object
 2   Peso                      79 non-null     object
 3   Tipo de moto              111 non-null    object
 4   Llantas                   64 non-null     object
 5   Transmisión               108 non-null    object
 6   Capacidad de combustible  106 non-null    object
 7   Suspensión delantera      102 non-null    object
 8   Suspensión trasera        102 non-null    object
 9   Freno delantero           111 non-null    object
 10  Freno trasero             111 non-null    object
 11  Neumático delantero       91 non-null     object
 12  Neumático trasero         91 non-null     object
 13  Tipo de motor             108 non-null    object
 14  Largo de moto             

Veamos la forma de nuestro dataset generado mediante webscraping:

In [14]:
motos.shape

(130, 33)

Vemos que contamos con 130 elementos con 33 características cada uno.

Quitamos las columnas de nombre 'Bloque *' para facilitar la visualización de otros atributos:

In [24]:
pattern_to_remove = 'Bloque'

filtered_columns = [col for col in motos.columns if pattern_to_remove not in col]

In [25]:
motos = motos[filtered_columns]

Veamos un resúmen de los primeros 50 elementos recolectados

In [37]:
motos.head(50)

,Nombre,Modelo,Peso,Tipo de moto,Llantas,Transmisión,Capacidad de combustible,Suspensión delantera,Suspensión trasera,Freno delantero,...,Color,Alimentación,Largo de caja,Ancho de caja,Alto de caja,Freno delantero Texto,Freno Trasero Texto,Garantia,Origen,Relación
0,Moto Honda Wave 110 S - 2024,Honda Wave 110 S - 2024,101 kg,Cub,Rayos,4 velocidades,"3,7 litros",Horquilla telescópica,Basculante de doble brazo con doble amortiguad...,Tambor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Moto Honda Wave 110 Cast Disk - 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Moto Honda Xr 250 New Tornado - 2024,Honda Xr 250 New Tornado - 2024,150 kg,ON-OFF,Rayos,6 velocidades,"11,5 litros",Horquilla telescópica,Pro-Link,Disco,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Moto Bajaj Rouser 200 Ns - 2023,NaN,145 Kg,Naked,Aleación,6 velocidades,10 litros,Horquilla telescópica anti fricción,Monoamortiguador Nitrox,Disco ABS,...,Negro,Inyección electrónica,2.017,804,1.195,NaN,NaN,NaN,NaN,NaN
4,Moto Motomel Blitz 110 (R/T) - 2022,NaN,99 KG,Cub,Rayos,4 VELOCIDADES,"3,5 LTS",HORQUILLA HIDRA´ULICA,AMORTIGUADORES HIDRA´ULICOS,TAMBOR / DISCO,...,BlancoNegro,NaN,NaN,NaN,NaN,TAMBOR / DISCO,TRASERO TAMBOR,NaN,NaN,NaN
5,Moto Honda Xr 150 L - 2024,Honda Xr 150 L - 2024,150 kg,ON-OFF,Rayos,5 velocidades,12 litros,Horquilla telescópica,Brazo oscilante con monoamortiguador,Disco,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Moto Motomel Skua 150 Silver Edition - 2022,2022,123 Kg,ON-OFF,Rayos,6 velocidades,"11,5 litros",Horquilla telescópica invertida,Monoamortiguador,Disco,...,Gris,Carburador,NaN,NaN,NaN,NaN,NaN,7 meses o 6.000 km,Argentina,NaN
7,Moto Gilera Smash Base 110 - 2023,NaN,100 kg,Cub,Rayos,4 velocidades con embrague automático,"4,1 litros",Horquilla telescópica,Doble amortiguador,Tambor,...,Rojo,Carburador,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Moto Benelli 180 S - 2023,NaN,147 kg,Naked,Aleación,Mecánica 6 v,10 Lts,Horquilla telescópica invertidas,Monoshock Central,Disco Ventilado,...,BlancoNegroRojo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Moto Siam Qu 110 R/T - 2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
